In [ ]:
'''
Notes on confidence interval calculation:
FIM assumes local linearity near the optimum and yields uncertainty (i.e., standard deviations) around the point estimate, not the true 
mean or median of a parameter distribution. Therefore, the confidence intervals are correctly computed here because the code below gives 
uncertainty ranges centered on those fitted values.
'''

In [1]:
# 95% CI from Monte Carlo
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2e}'.format)

# Data import
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtl/PlasticFADE.xlsx"  # CHECK: confirm file path
sheet_name = "Uncertainty"
data_CI = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI = data_CI.iloc[36:42] # Row index minus 3, change this range for other polymers
print(data_CI)

# Parameter estimates
a_i, delta_i, b_i, alpha_i, c_i, beta_i = data_CI.iloc[:, 2].values
# Standard deviations of parameters
a_i_std, delta_i_std, b_i_std, alpha_i_std, c_i_std, beta_i_std = data_CI.iloc[:, 3].values

             Process Parameter  Estimate Standard deviation
36  PP fragmentation       a_i  4.66e-06           1.85e-05
37               NaN   delta_i  2.26e+00           1.23e+00
38               NaN       b_i  5.34e-04           1.53e-03
39               NaN   alpha_i  5.00e-13           9.01e-01
40               NaN       c_i  8.68e+00           9.42e-08
41               NaN    beta_i  1.44e+00           3.67e-01


In [3]:
# Input parameters
data_input = pd.read_excel(file_path, sheet_name="Results", usecols="A:E", skiprows=13)
data_input = data_input[data_input.iloc[:, 0] == "PP"]  # Change for different polymer types
print(data_input)
data_input.columns = ['Polymer', 'Compartment', 's', 'I_j', 'P_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []

# Loop through each row of input
for index, row in data_input.iterrows():
    s = row['s']
    I_j = row['I_j']
    P_j = row['P_j']

    # Monte Carlo with log-normal distribution
    a_i_samples = np.random.lognormal(np.log(a_i), a_i_std, N)
    delta_i_samples = np.random.lognormal(np.log(delta_i), delta_i_std, N)
    b_i_samples = np.random.lognormal(np.log(b_i), b_i_std, N)
    alpha_i_samples = np.random.lognormal(np.log(alpha_i), alpha_i_std, N)
    c_i_samples = np.random.lognormal(np.log(c_i), c_i_std, N)
    beta_i_samples = np.random.lognormal(np.log(beta_i), beta_i_std, N)
    
    # Impose caps to filter out physically meaningless values (in most empirical contexts, exponents above 4–5 are rare)
    delta_i_samples = np.clip(delta_i_samples, 0, 5)
    alpha_i_samples = np.clip(alpha_i_samples, 0, 5)
    beta_i_samples = np.clip(beta_i_samples, 0, 5)
    
    # Compute k_frag for each sample
    k_samples = a_i_samples * (s**delta_i_samples) * (b_i_samples * I_j**alpha_i_samples + c_i_samples * P_j**beta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples (good habit, especially when NaNs are found in the CIs)

    # 95% CI in log-space
    if np.all(k_samples == 0):
        lower_bound, upper_bound = 0, 0
    else:
        log_k = np.log10(k_samples[k_samples > 0])  # Exclude zeros (for subsoil & sediment)
        lower_bound = 10 ** np.percentile(log_k, 2.5)
        upper_bound = 10 ** np.percentile(log_k, 97.5)
        
    k_point = a_i * (s**delta_i) * (b_i * I_j**alpha_i + c_i * P_j**beta_i)
    '''   
    # If k_point is zero, force CI to be zero too
    if k_point == 0 or np.isnan(k_point):
        lower_bound = 0
        upper_bound = 0
    '''
    results.append({'Compartment': row['Compartment'], 'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})

# Display results
results_CI = pd.DataFrame(results)
print("\n", results_CI)
print(f"\n{N - len(k_samples)} out of {N} samples were invalid and removed.")

  Polymer (i) Compartment (j)  SA:V [cm-1]  I_j [W/m2]  P_j [mW]
0          PP             Air           25    1.00e+01  4.04e-03
1          PP         Topsoil           25    1.00e-01  1.15e-03
2          PP         Subsoil           25    0.00e+00  0.00e+00
3          PP           Beach           25    1.25e+01  2.64e-02
4          PP   Water surface           25    1.00e+01  3.30e-03
5          PP    Water column           25    0.00e+00  1.69e-06
6          PP        Sediment           25    0.00e+00  0.00e+00

      Compartment  k_point  CI_lower  CI_upper
0            Air 2.48e-05  1.13e-08  3.09e+00
1        Topsoil 7.08e-06  7.51e-09  1.04e+00
2        Subsoil 0.00e+00  0.00e+00  0.00e+00
3          Beach 3.17e-04  5.75e-08  1.55e+01
4  Water surface 1.94e-05  9.59e-09  2.70e+00
5   Water column 2.98e-10  7.34e-20  3.35e-03
6       Sediment 0.00e+00  0.00e+00  0.00e+00

0 out of 10000 samples were invalid and removed.


In [5]:
# Write confidence intervals back to Excel (without modifying)
import xlwings as xw

wb = xw.Book(file_path)
sheet = wb.sheets["Results"]

start_row = 15  # CHECK: Change this index for other polymers
sheet.range(f'K{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'L{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)

wb.save()
wb.close()